In [43]:
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import auth
    # auth.authenticate_user() # commented out the previous authentication method

    # Authenticate using the service account file
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cred.json'
    print("Authenticated using cred.json")

Authenticated using cred.json


In [2]:
CREDENTIALS_PATH = "/content/cred.json"

In [3]:
# Import the validation function from your code
import os
import json

# Using the validate_service_account function from your code
def validate_service_account(cred_path):
    """
    Validates the service account credentials file and prints its contents safely
    """
    try:
        # Check if file exists
        if not os.path.exists(cred_path):
            return False, f"Credential file not found at {cred_path}"

        # Load and validate JSON format
        with open(cred_path, 'r') as f:
            cred_data = json.load(f)

        # Check for required fields
        required_fields = [
            'type', 'project_id', 'private_key_id',
            'private_key', 'client_email', 'client_id'
        ]

        missing_fields = [field for field in required_fields if field not in cred_data]
        if missing_fields:
            return False, f"Missing required fields in credentials: {', '.join(missing_fields)}"

        # Print safe information (not the private key)
        safe_info = {
            'type': cred_data.get('type'),
            'project_id': cred_data.get('project_id'),
            'client_email': cred_data.get('client_email'),
            'auth_uri': cred_data.get('auth_uri'),
            'token_uri': cred_data.get('token_uri'),
        }

        return True, safe_info

    except json.JSONDecodeError:
        return False, "Invalid JSON format in credential file"
    except Exception as e:
        return False, f"Error validating credentials: {str(e)}"

# Validate the credentials
is_valid, info = validate_service_account(CREDENTIALS_PATH)
if is_valid:
    print("✅ Credentials are valid!")
    print("Credential info:")
    for key, value in info.items():
        print(f"  - {key}: {value}")
else:
    print("❌ Invalid credentials:", info)

❌ Invalid credentials: Credential file not found at /content/cred.json


In [46]:
# Set environment variable for Google Cloud authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS_PATH

# Import the initialization function from your code
from google.cloud import aiplatform
from google.oauth2 import service_account

# Using the initialize_vertex_ai function from your code
def initialize_vertex_ai(cred_path, project_id=None, location="us-central1"):
    """
    Initialize Vertex AI with the given credentials
    """
    try:
        # If project_id is not provided, extract it from credentials
        if not project_id:
            with open(cred_path, 'r') as f:
                cred_data = json.load(f)
                project_id = cred_data.get('project_id')

        # Create credentials object
        credentials = service_account.Credentials.from_service_account_file(
            cred_path,
            scopes=["https://www.googleapis.com/auth/cloud-platform"]
        )

        # Initialize Vertex AI
        aiplatform.init(
            project=project_id,
            location=location,
            credentials=credentials
        )

        return True, f"Successfully initialized Vertex AI for project: {project_id} in {location}"

    except Exception as e:
        return False, f"Error initializing Vertex AI: {str(e)}"

# Initialize Vertex AI
init_success, message = initialize_vertex_ai(CREDENTIALS_PATH)
print(message)

Successfully initialized Vertex AI for project: project-627b8b21-5a0c-430f-aa1 in us-central1


In [51]:
MODEL_ID="gemini-2.5-pro"
PROJECT_ID="project-627b8b21-5a0c-430f-aa1"
CREDENTIALS_PATH = "/content/cred.json"


In [53]:
from google import genai

client = genai.Client(api_key=GEMINI_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Tell me a joke"
)
print(response.text)

Why don't scientists trust atoms?

Because they make up everything!


In [54]:
from openai import OpenAI

client = OpenAI(
    api_key=GEMINI_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = client.chat.completions.create(
  model="gemini-2.0-flash",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)

for chunk in response:
    print(chunk.choices[0].delta)

ChoiceDelta(content='Hello', function_call=None, refusal=None, role='assistant', tool_calls=None)
ChoiceDelta(content='!', function_call=None, refusal=None, role='assistant', tool_calls=None)
ChoiceDelta(content=' How can I help you today?\n', function_call=None, refusal=None, role='assistant', tool_calls=None)


In [49]:
from google import genai
from google.genai import types
client = genai.Client(
  vertexai=True, project=PROJECT_ID, location="global",
)
# If your image is stored in Google Cloud Storage, you can use the from_uri class method to create a Part object.
IMAGE_URI = "gs://generativeai-downloads/images/scones.jpg"
model = "gemini-2.5-pro"
response = client.models.generate_content(
  model=model,
  contents=[
    "What is shown in this image?",
    types.Part.from_uri(
      file_uri=IMAGE_URI,
      mime_type="image/png",
    ),
  ],
)
print(response.text, end="")

This is a beautifully styled, top-down photograph of a rustic and inviting breakfast or snack scene. Here's a breakdown of what's shown:

*   **Main Food Item:** The focus is on several round, crumbly baked goods, which appear to be **blueberry scones or biscuits**. They are sprinkled with sugar and arranged on a piece of white parchment paper that is stained with purple from the baked berries.
*   **Accompaniments:**
    *   A small dark bowl filled with fresh **blueberries**.
    *   A decorative silver spoon resting by the bowl, engraved with the words **"LET'S JAM"**.
    *   Two white mugs of **coffee**: one appears to be black coffee, and the other contains coffee with milk or cream.
*   **Decorative Elements:**
    *   A bouquet of fresh **pink peonies** on the right side of the frame.
    *   A few scattered fresh blueberries and small mint leaves used as garnish.
*   **Setting:** The entire scene is set on a dark, textured surface, which gives it a moody and rustic feel. The c